In [1]:
!nvidia-smi


Mon Jun 16 19:07:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             28W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Import Lib

In [ ]:
!pip install "mineru[core]>=2.0.0
!apt-get -y install poppler-utils
%pip install google-genai
%pip install streamlit
# %pip -q install huggingface_hub rank_bm25 httpx
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai langchain-groq
%pip install -Uq python_dotenv
%pip install -qU langchain-ollama
%pip install -qU rank_bm25 httpx
%pip install python-pptx
%pip install gradio pymupdf


/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-dev libpoppler-private-dev libpoppler118


In [ ]:
!mineru-models-download --source huggingface --model_type pipeline

In [ ]:
!mineru -p /kaggle/input/attention/attention_scan.pdf -o /kaggle/working/ --source local

## Extract file PDF

In [ ]:
%%writefile extract_pdf.py
import os
import shutil
import subprocess

def extract_pdf(pdf_file_path: str):
    """
    Extract content from a PDF file using minerU CLI and restructure outputs
    to match legacy structure.

    Args:
        pdf_file_path (str): Path to the input PDF file.
        pdf_filename (str): Name of the PDF file (without .pdf).
        output_base (str): Base directory to save outputs.
    """
    # === 1️⃣ Call minerU CLI ===
    cmd = f"mineru -p {pdf_file_path} -o /kaggle/working --source local"
    print(f"[INFO] Running: {cmd}")
    subprocess.run(cmd, shell=True, check=True)


In [ ]:
# %%writefile extract_pdf.py
# import os
# from magic_pdf.data.data_reader_writer import FileBasedDataWriter, FileBasedDataReader
# from magic_pdf.data.dataset import PymuDocDataset
# from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze
# from magic_pdf.config.enums import SupportedPdfParseMethod

# def extract_pdf(pdf_file_path: str, pdf_filename: str, output_base: str = "/kaggle/working/output"):
#     """
#     Extract content from a PDF file using minerU and save markdown and middle JSON results.

#     Args:
#         pdf_file_path (str): Path to the input PDF file.
#         output_base (str): Base directory to save outputs (images, jsons).
#     """

#     # Prepare output directories
#     images_dir = os.path.join(output_base, "images")
#     jsons_dir = os.path.join(output_base, "jsons")
#     local_image_dir = os.path.join(images_dir, pdf_filename)
#     local_md_dir = os.path.join(jsons_dir, pdf_filename)
#     image_dir = os.path.basename(local_image_dir)

#     os.makedirs(local_image_dir, exist_ok=True)
#     os.makedirs(local_md_dir, exist_ok=True)

#     # Create writers
#     image_writer = FileBasedDataWriter(local_image_dir)
#     md_writer = FileBasedDataWriter(local_md_dir)

#     # Read PDF content
#     reader1 = FileBasedDataReader("")
#     pdf_bytes = reader1.read(pdf_file_path)

#     # Create Dataset instance
#     ds = PymuDocDataset(pdf_bytes)

#     # Run inference and postprocess
#     if ds.classify() == SupportedPdfParseMethod.OCR:
#         infer_result = ds.apply(doc_analyze, ocr=True)
#         pipe_result = infer_result.pipe_ocr_mode(image_writer)
#     else:
#         infer_result = ds.apply(doc_analyze, ocr=False)
#         pipe_result = infer_result.pipe_txt_mode(image_writer)

#     # Dump markdown and middle JSON
    
#     ### draw layout result on each page
#     pipe_result.draw_layout(os.path.join(local_md_dir, f"{pdf_filename}_layout.pdf"))
  

#     ### dump content list
#     pipe_result.dump_content_list(md_writer, f"{pdf_filename}_content_list.json", image_dir)

#     ### dump markdown
#     md_filename = f"{pdf_filename}.md"
#     pipe_result.dump_md(md_writer, md_filename, image_dir)

#     ### dump middle JSON
#     middle_json_filename = f"{pdf_filename}_middle.json"
#     pipe_result.dump_middle_json(md_writer, middle_json_filename)

#     print(f"[INFO] Extracted and saved to {local_md_dir}")



## Prompt Builder

In [ ]:
%%writefile prompt_builder.py
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from PIL import Image
from google import genai
# Khởi tạo client với API key
client = genai.Client(api_key="YOUR_GEMINI_API_KEY")


def summarize_image_with_gemini(image_path, caption):
    with Image.open(image_path) as image:
        prompt = (
            f"The following image is from a scientific paper, and the caption if existed is:\n\n"
            f"{caption}\n\n"
            "Based on both the visual content and the caption if existed, write a detailed but concise description of the image. "
            "Keep the description within 100 words maximum. Use a neutral academic tone.\n\n"
            "Then, generate a very short image caption (one single sentence, 5–15 words) for displaying below the image in a slide. "
            "The short caption must be included even if you are unsure. Do not skip it.\n\n"
            "Return the result in the following format exactly:\n"
            "**Image Description**: <your detailed description here>\n"
            "**Image Caption**: <your short caption here>"
        )

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[image, prompt]
        )
        return response.text.strip()
def summarize_table_with_gemini(caption, table_html):
    prompt = (
        f"You are an assistant tasked with analyzing and summarizing tables from scientific papers. "
        f"The following is an HTML table and its caption (if existed).\n\n"
        f"Table Caption: {caption}\n"
        f"Table Data (HTML):\n{table_html}\n\n"
        "Analyze the table and provide a concise summary of the main results, explicitly including key numerical values. "
        "Highlight which methods performed better on each dataset (TotalText and CTW-1500) based on recall (R), precision (P), and H-mean (H). "
        "When identifying the best-performing methods, include the actual metric values (e.g., 'CRAFT achieved the highest recall of 85.3% on CTW-1500'). "
        "Use a clear and objective academic tone. Do not restate the caption.\n\n"
        "After the description, generate a a **very short, concise table caption (one single sentence, 5–15 words) that clearly summarizes the key insight of the table.\n\n"
        "Return the result in the following format:\n"
        "**Table Description**: <your detailed table analysis here>\n"
        "**Table Caption**: <your concise caption here>"
    )
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt]
    )
    return response.text.strip()


### Prompt summarize slide ###

def prompt_ver1(text, equations):

    eq_list = "\n".join([
        f"- {eq.get('equation_markdown', '')} (filename: {eq.get('src', '')}"
        for eq in equations
    ]) if equations else "[None]"

    prompt = f"""
        You are a scientific assistant helping to summarize and organize technical research documents.

        Your task:

        1. Summarize the content into **no more than 5 short bullet points**:
        • Each bullet must start with the • symbol (U+2022).
        • Use plain English and write **short, concise, clear sentences**.
        • Focus only on the **main contributions, methods, or findings**.
        • Avoid redundancy. Each point should convey a unique idea.
        • Only write as many bullets as needed — do not force 5 bullets if the content is limited.

        2. Analyze the equations listed below:
        • If a bullet point refers to or is supported by an equation, add the filename in this format: (Equation: filename.png) at the end of that bullet.
        • Do NOT include math notation or equation content in the bullets.
        • Only annotate relevant equations.

        3. Make sure the summary reflects all key insights from the text and equations.
        • Do NOT refer to images or tables explicitly.
        • Use plain English. Avoid any LaTeX, symbols, or code.

        **Output Format (strict):**
        • Bullet 1  
        • Bullet 2  
        (up to 5 bullets)

        Do NOT include any titles, explanations, or extra text.

        ---

        **Raw Text:**
        {text.strip()}

        ---

        **Equations:**
        {eq_list}

        Write concise summary bullets now, annotated with equations where appropriate.
        """
    return prompt


def prompt_ver2(text, equations, images):
    import os

    eq_list = "\n".join([
        f"- {eq.get('equation_markdown', '')} (filename: {eq.get('src', '')}"
        for eq in equations
    ]) if equations else "[None]"

    max_section = len(images) + 1

    figure_list = []
    table_list = []
    for img in images:
        alt = img.get("alt", "")
        name = img.get("src", "")
        caption = img.get("caption", "").strip()
        item_str = f"(Caption: {caption}) - (Filename: {name})"
        if alt.lower() == "table":
            table_list.append(item_str)
        else:
            figure_list.append(item_str)

    img_list = "\n".join(figure_list) if figure_list else "[None]"
    table_list_str = "\n".join(table_list) if table_list else "[None]"

    prompt = f"""
        You are a scientific assistant helping to summarize and organize technical research documents.

        Your task is as follows:

        **Input:**
        - A raw text block from a scientific paper (see below).
        - A list of related equations in LaTeX/Markdown format (with filenames).
        - A list of related figures and tables (each with a caption and filename).

        ---

        **Your Goals:**

        1. **Summarize** the input text into concise, clear bullet points.
        2. **Organize** the bullets into logical sections:
        - Each section should have a header: `### Section N: Title of section`
        - Each section can have **up to 5 bullets**.
        - Each section can reference **at most 2 visuals** (images or tables).
        - The **total number of sections must not exceed {max_section}**.

        3. **Annotate** each bullet with any clearly related visual or equation:
        - Use format: `(Image: figure1.png, figure2.jpg) or (Table: table1.png) or (Equation: 1.png)`
        - Use at most 2 references per bullet.
        - If no relevant visual/equation applies, leave it untagged.
        

        **Coverage Requirement:**

        - Make sure that **each image, table, and equation is referenced by at least one bullet** if relevant.
        - Distribute references across sections so that **no image/table/equation is left out unnecessarily**.
        - Avoid forcing unrelated visuals into bullets — only include them where they naturally fit.


        **Formatting Guidelines:**
        - Use natural language to explain all ideas — do NOT include any mathematical notation, LaTeX, or special symbols (like $...$)
        - Each bullet must be a standalone factual summary.
        - Do not reuse or rephrase raw text — rewrite clearly.
        - Do not invent filenames or captions.
        - Do not add more than {max_section} sections.


        **Raw Text:**
        {text.strip()}

        ---

        **Equations:**
        {eq_list}

        ---

        **Figures:**
        {img_list}

        ---

        **Tables:**
        {table_list_str}

        ---

        Now write the summary bullets, organized by section, and annotate related visuals/equations accordingly.
        """
    
    return prompt



 

## Process Json Data

In [ ]:
%%writefile process_json_data.py
import json
import os
from io import BytesIO
from PIL import Image
from google import genai
import uuid
from langchain.schema import Document
from prompt_builder import summarize_image_with_gemini, summarize_table_with_gemini

def split_chunks_by_title(data, max_chunk_len=1000):
    chunks = []
    current_title = ""
    current_chunk_texts = []
    i = 0

    def get_text_len(text_list):
        return sum(len(t.strip()) for t in text_list)

    def flush_chunk():
        nonlocal current_chunk_texts
        if not current_chunk_texts:
            return

        full_text = "\n".join(current_chunk_texts).strip()
        if not full_text:
            return

        # Nếu là phần Reference, chia nhỏ theo dòng
        if "reference" in current_title.lower() or "tài liệu tham khảo" in current_title.lower():
            lines = full_text.split("\n")
            buffer = []
            buffer_len = 0
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                if buffer_len + len(line) > max_chunk_len:
                    chunks.append(f"{current_title.strip()}\n" + "\n".join(buffer).strip())
                    buffer = [line]
                    buffer_len = len(line)
                else:
                    buffer.append(line)
                    buffer_len += len(line)
            if buffer:
                chunks.append(f"{current_title.strip()}\n" + "\n".join(buffer).strip())
        else:
            chunks.append(f"{current_title.strip()}\n{full_text}")

        current_chunk_texts.clear()

    while i < len(data):
        entry = data[i]
        temp_texts = []

        # Nếu là tiêu đề
        if entry["type"] == "text" and entry.get("text_level") == 1:
            # Gộp các tiêu đề liên tiếp
            new_title = entry["text"].strip()
            i += 1
            while i < len(data) and data[i].get("text_level") == 1:
                new_title += "\n" + data[i]["text"].strip()
                i += 1
            flush_chunk()
            current_title = new_title
            continue

        # Xử lý phương trình
        if entry["type"] == "equation":
            if i > 0 and data[i-1]["type"] == "text":
                prev_text = data[i-1]["text"].strip()
                if not current_chunk_texts or current_chunk_texts[-1] != prev_text:
                    temp_texts.append(prev_text)
            temp_texts.append(entry["text"].strip())
            if i + 1 < len(data) and data[i+1]["type"] == "text":
                temp_texts.append(data[i+1]["text"].strip())
                i += 1

        elif entry["type"] == "text":
            text = entry["text"].strip()
            if text:
                temp_texts.append(text)

        current_chunk_texts.extend(temp_texts)

        # Nếu chunk hiện tại vượt quá max length
        if get_text_len(current_chunk_texts) > max_chunk_len:
            j = i + 1
            while j < len(data):
                next_entry = data[j]
                if next_entry.get("text_level") == 1:
                    break

                lookahead_temp = []

                if next_entry["type"] == "equation":
                    if j > 0 and data[j-1]["type"] == "text":
                        prev_text = data[j-1]["text"].strip()
                        if not current_chunk_texts or current_chunk_texts[-1] != prev_text:
                            lookahead_temp.append(prev_text)
                    lookahead_temp.append(next_entry["text"].strip())
                    if j + 1 < len(data) and data[j+1]["type"] == "text":
                        lookahead_temp.append(data[j+1]["text"].strip())
                        j += 1
                elif next_entry["type"] == "text":
                    lookahead_temp.append(next_entry["text"].strip())

                if get_text_len(lookahead_temp) + get_text_len(current_chunk_texts) <= max_chunk_len:
                    current_chunk_texts.extend(lookahead_temp)
                    j += 1
                    i = j
                else:
                    break

            flush_chunk()

        i += 1

    flush_chunk()
    return chunks

def process_json_data(content_list_json, images_root):
    with open(content_list_json, "r", encoding="utf-8") as f:
        data = json.load(f)

    chunks = split_chunks_by_title(data)

    image_summaries = []
    image_captions = []
    for item in data:
        if item['type'] == 'image':
            full_path = os.path.join(images_root, item['img_path'])
            caption = item.get('img_caption', '')
            try:
                response = summarize_image_with_gemini(full_path, caption)
                # Tách phần mô tả và caption từ response
                description = ""
                short_caption = ""
                for line in response.splitlines():
                    if line.startswith("**Image Description**:"):
                        description = line.replace("**Image Description**:", "").strip()
                    elif line.startswith("**Image Caption**:"):
                        short_caption = line.replace("**Image Caption**:", "").strip()
                image_summaries.append(description)
                image_captions.append(short_caption)
            except Exception as e:
                image_summaries.append(str(e))
                image_captions.append("")

    table_summaries = []
    table_captions = []
    for item in data:
        if item['type'] == 'table':
            caption = item.get('table_caption', '')
            html = item['table_body']
            try:
                response = summarize_table_with_gemini(caption, html)
                # Tách phần mô tả và caption từ response
                description = ""
                short_caption = ""
                for line in response.splitlines():
                    if line.startswith("**Table Description**:"):
                        description = line.replace("**Table Description**:", "").strip()
                    elif line.startswith("**Table Caption**:"):
                        short_caption = line.replace("**Table Caption**:", "").strip()
                table_summaries.append(description)
                table_captions.append(short_caption)
            except Exception as e:
                table_summaries.append(str(e))
                table_captions.append("")

    # 3. Hàm chuẩn bị documents từ summaries
    def create_summary_documents(summaries, data_type):
        return [
            Document(page_content=summary, metadata={"type": data_type, "doc_id": str(uuid.uuid4())})
            for summary in summaries
        ]

    # Chuyển đổi summaries thành documents
    text_summary_docs = create_summary_documents(chunks, "text_summary")
    image_summary_docs = create_summary_documents(image_summaries, "image_summary")
    table_summary_docs = create_summary_documents(table_summaries, "table_summary")
    all_docs_summary = text_summary_docs + image_summary_docs + table_summary_docs

    return all_docs_summary,image_captions,table_captions

## Reranker

In [ ]:
%%writefile reranker.py
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load model
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")
model = AutoModelForSequenceClassification.from_pretrained("BAAI/bge-reranker-v2-m3").to(device).eval()


def get_top_k_docs(query, docs, k=1):
    documents = [doc["document"] if isinstance(doc, dict) else str(doc) for doc in docs]

    if not query or not documents:
        return []

    scores = []
    batch_size = 4  # dùng k làm batch size

    with torch.no_grad():
        for i in range(0, len(documents), batch_size):
            batch_docs = documents[i:i + batch_size]
            pairs = [[query, doc] for doc in batch_docs]
            inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors="pt").to(device)
            logits = model(**inputs, return_dict=True).logits.view(-1)
            scores.extend(logits.tolist())

    results = [{"document": doc, "score": float(round(score, 4))} for doc, score in zip(documents, scores)]
    top_k_docs = sorted(results, key=lambda x: x["score"], reverse=True)[:k]
    return top_k_docs


## BGE-M3 Embeddings

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
# Khởi tạo embedding
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

## Create Slides

In [ ]:
%%writefile create_slide.py
import time
import json
import shutil
import re
import os
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pdf2image import convert_from_path
from PIL import Image
from prompt_builder import prompt_ver1, prompt_ver2
from google import genai
# Khởi tạo client với API key
client = genai.Client(api_key="YOUR_GEMINI_API_KEY")

import os
import shutil
import json

def copy_unique_equation_images(pdf_name, middle_json, output_root="/kaggle/working/"):
    
    save_dir = os.path.join(output_root, pdf_name, "auto", "images")
    os.makedirs(save_dir, exist_ok=True)

    with open(middle_json, "r", encoding="utf-8") as f:
        data = json.load(f)

    image_paths = []

    for page in data.get("pdf_info", []):
        # Các tool thường gọi là "blocks" hoặc "preproc_blocks"
        blocks = page.get("blocks", []) + page.get("preproc_blocks", [])
        for block in blocks:
            if block.get("type") == "interline_equation":
                lines = block.get("lines", [])
                for line in lines:
                    spans = line.get("spans", [])
                    for span in spans:
                        if span.get("type") == "interline_equation":
                            image_path = span.get("image_path")
                            if image_path:
                                image_paths.append(image_path)

    print(f"🔍 Found {len(image_paths)} raw image paths in {middle_json}")
    unique_image_paths = list(dict.fromkeys(image_paths))
    print("Image paths after deduplication:", unique_image_paths )

    for idx, img_path_raw in enumerate(unique_image_paths):
        imgpath = os.path.join(save_dir, img_path_raw)
        if os.path.exists(img_path):
            dst = os.path.join(save_dir, f"{idx}.png")
            shutil.copy(img_path, dst)
            print(f"✅ Copied: {img_path} -> {dst}")
        else:
            print(f"⚠️ Not found: {img_path}")

    print(f"🎉 Done! Total unique equations copied: {len(unique_image_paths)}")




def parse_markdown(filepath, content_list_json, pdf_filename, image_captions, table_captions):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()

    # 1. Đọc danh sách bảng từ JSON
    with open(content_list_json, 'r', encoding='utf-8') as f:
        raw_table_data = json.load(f)

    table_images = []
    table_idx = 0
    for item in raw_table_data:
        if item.get("type") == "table":
            path = item.get("img_path", "")
            # Sử dụng short caption từ table_captions
            short_caption = table_captions[table_idx] if table_idx < len(table_captions) else ""
            caption = f"Table {table_idx + 1}. {short_caption}" if short_caption else f"Table {table_idx + 1}"
            table_images.append({"path": path, "caption": caption})
            table_idx += 1

    global_figures = []
    global_tables = table_images

    header_pattern = re.compile(r'(#{1,4})\s+(.*)')
    blocks = []
    current_block = None

    lines = content.splitlines()
    i = 0
    figure_idx = 0
    while i < len(lines):
        line = lines[i]
        header_match = header_pattern.match(line)

        if header_match:
            if current_block:
                blocks.append(current_block)
            level = len(header_match.group(1))
            title = header_match.group(2).strip()
            current_block = {
                'heading': title,
                'level': level,
                'text': '',
                'equations': [],
                'images': []
            }
        elif current_block:
            # Ảnh markdown
            img_match = re.match(r'!\[(.*?)\]\((.*?)\)', line)
            if img_match:
                alt, src = img_match.groups()
                # Caption ưu tiên từ image_captions
                short_caption = image_captions[figure_idx] if figure_idx < len(image_captions) else ""
                caption = f"Figure {figure_idx + 1}. {short_caption}" if short_caption else f"Figure {figure_idx + 1}"
                figure_info = {'alt': alt, 'src': src, 'caption': caption}
                current_block['images'].append(figure_info)
                global_figures.append(figure_info)
                figure_idx += 1
            elif '<table' in line:
                if len(global_tables) > 0:
                    table_info = global_tables[len(global_tables) - len(table_images)]
                    current_block['images'].append({
                        'alt': 'table',
                        'src': table_info['path'],
                        'caption': table_info['caption']
                    })
                    table_images.pop(0)
            else:
                current_block['text'] += line + '\n'
        i += 1

    if current_block:
        blocks.append(current_block)

    # 3. Equation → lưu path ảnh + markdown gốc
    eq_counter = 0
    for block in blocks:
        matches = list(re.finditer(r'\$\$(.*?)\$\$', block['text'], re.DOTALL))
        block['equations'] = []
        for match in matches:
            eq_text = match.group(0)
            eq_name = f"{eq_counter}.png"
            eq_path = os.path.join(os.path.join("/kaggle/working/", pdf_filename, "auto/equations"), eq_name)
            block['equations'].append({
                "src": eq_path,
                "equation_markdown": eq_text
            })
            eq_counter += 1

    # 4. Thêm ảnh và bảng được tham chiếu trong text
    fig_ref_pattern = re.compile(r'(?:Fig\.|Figure)\s*\.?\s*(\d+)', re.IGNORECASE)
    table_ref_pattern = re.compile(r'Table\s*\.?\s*(\d+)', re.IGNORECASE)

    for block in blocks:
        referenced = set()

        for fig_match in fig_ref_pattern.findall(block['text']):
            index = int(fig_match) - 1
            if 0 <= index < len(global_figures):
                fig_info = global_figures[index]
                fig_id = (fig_info['src'], fig_info['caption'])
                if fig_id not in referenced:
                    block['images'].append(fig_info)
                    referenced.add(fig_id)

        for table_match in table_ref_pattern.findall(block['text']):
            index = int(table_match) - 1
            if 0 <= index < len(global_tables):
                table_info = global_tables[index]
                table_data = {
                    'alt': 'table',
                    'src': table_info['path'],
                    'caption': table_info['caption']
                }
                table_id = (table_data['src'], table_data['caption'])
                if table_id not in referenced:
                    block['images'].append(table_data)
                    referenced.add(table_id)

        # 5. Loại bỏ trùng lặp theo `src`
        seen = set()
        unique_images = []
        for img in block['images']:
            key = img.get('src')
            if key and key not in seen:
                unique_images.append(img)
                seen.add(key)
        block['images'] = unique_images
    print("Image cap:", image_captions)   
    return blocks

def update_levels_from_heading_text(blocks):
    cleaned_blocks = []

    for block in blocks:
        heading = block['heading'].strip()
        # Loại các block: abstract, references, appendix (dưới mọi hình thức)
        heading_lower = heading.lower()

        # Bỏ qua abstract, references, appendix, appendices, appendix a, etc.
        if (
            heading_lower in {"abstract", "references", "appendix", "appendices"} or
            heading_lower.startswith("appendix") or
            heading_lower.startswith("a.")
        ):
            continue

        # Nếu gặp "Conclusion" thì xử lý block này rồi dừng
        if "conclusion" in heading.lower():
            # Cập nhật level nếu có đánh số
            match = re.match(r'^(\d+(\.\d+)*)', heading)
            if match:
                block['level'] = match.group(1).count('.') + 1
            else:
                block['level'] = 1

            cleaned_blocks.append(block)
            break  # Dừng sau khi xử lý conclusion

        # Cập nhật level nếu có đánh số
        match = re.match(r'^(\d+(\.\d+)*)', heading)
        if match:
            block['level'] = match.group(1).count('.') + 1
        else:
            block['level'] = 1

        cleaned_blocks.append(block)

    return cleaned_blocks


### Create Content Slide ###
def create_content_slide(blocks):
    content_lines = []
    for block in blocks:
        level = block['level']
        heading = block['heading'].strip()

        # Loại bỏ tiền tố "1 " nếu có do lỗi Markdown parser
        heading = re.sub(r'^1\s+', '', heading)

        indent = '          ' * (level - 1) # mỗi level thụt vào 2 space
        content_lines.append(f"{indent}{heading}")

    return {
        'heading': 'Content',
        'level': 1,
        'text': '\n'.join(content_lines),
        'equations': [],
        'images': []
    }

### Parse into sub sections ###
def parse_summary_into_sections(summarized_text, block):
    sections = []
    current_section = None
    current_bullets = []
    image_filenames = set()

    image_lookup = {img["src"]: img for img in block.get("images", [])}

    lines = summarized_text.strip().splitlines()
    heading = block["heading"]

    for line in lines:
        line = line.strip()
        if line.startswith("### Section"):
            # Lưu section cũ
            if current_section and current_bullets:
                sections.append({
                    "heading": heading,
                    "sub_heading": current_section,
                    "text": "\n".join(current_bullets).strip(),
                    "images": [
                        image_lookup[fname]
                        for fname in sorted(image_filenames)
                        if fname in image_lookup
                    ]
                })

            # Reset cho section mới
            current_section = line.replace("###", "").strip()
            current_bullets = []
            image_filenames = set()

        elif line.startswith("*"):
            content = line[1:].strip()

            # Tìm ảnh/table liên quan
            related_imgs = re.findall(r"\(Image: ([^)]+)\)", content)
            related_tables = re.findall(r"\(Table: ([^)]+)\)", content)
            image_filenames.update(related_imgs + related_tables)

            # Loại bỏ phần liên quan image/table khỏi nội dung
            clean_content = re.sub(r"\(Image: [^)]+\)", "", content)
            clean_content = re.sub(r"\(Table: [^)]+\)", "", clean_content).strip()

            current_bullets.append(clean_content)

    # Lưu section cuối
    if current_section and current_bullets:
        sections.append({
            "heading": heading,
            "sub_heading": current_section,
            "text": "\n".join(current_bullets).strip(),
            "images": [
                image_lookup[fname]
                for fname in sorted(image_filenames)
                if fname in image_lookup
            ]
        })

    return sections

def summarize_block(block):
    """
    Summarize a single block using either prompt_ver1 or prompt_ver2 based on number of images.
    Returns a list of summarized blocks.
    """
    text = block['text'].strip()
    if not text:
        print(f"Block ('{block['heading']}') is empty, skipping.")
        return []

    print(f"\n⏳ Summarizing block — {block['heading'][:30]}...")
    block_summaries = []

    # Count number of images
    num_images = len(block.get('images', []))
    num_equations = len(block.get('equations', []))
    
    try:
        if num_images > 2 or num_equations > 3:
            # Use prompt_ver2 for blocks with >2 images
            prompt = prompt_ver2(
                text=text,
                equations=block.get("equations", []),
                images=block.get("images", [])
            )
            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt
            )
            summarized = response.text.strip()
            
            # Split into sections
            sections = parse_summary_into_sections(summarized, block)
            block_summaries.extend(sections)
            
        else:
            # Use prompt_ver1 for blocks with <=2 images
            prompt = prompt_ver1(
                text=text,
                equations=block.get("equations", [])
            )
            
            response = client.models.generate_content(
                model="gemini-2.0-flash", 
                contents=prompt
            )
            time.sleep(2)
            summarized = response.text.strip()
            
            # Create a single block with the summary
            summary_block = {
                "heading": block["heading"],
                "text": summarized,
                "equations": block.get("equations", []),
                "images": block.get("images", [])
            }
            block_summaries.append(summary_block)

        print(f"✅ Done summarizing block")
        return block_summaries

    except Exception as e:
        print(f"❌ Error summarizing block: {e}")
        return [{
            "heading": block["heading"],
            "text": "[Summary failed]",
            "equations": block.get("equations", []),
            "images": block.get("images", [])
        }]
    
def summarize_blocks(blocks, start_idx=0, max_blocks=None, sleep_time=1.5):
    """
    Summarize multiple blocks using the summarize_block function.
    Returns a list of all summarized blocks.
    """
    end_idx = len(blocks) if max_blocks is None else start_idx + max_blocks
    all_summarized_blocks = []

    for i in range(start_idx, end_idx):
        block = blocks[i]
        summarized_blocks = summarize_block(block)
        all_summarized_blocks.extend(summarized_blocks)
        time.sleep(sleep_time)

    return all_summarized_blocks

### Create Slide ###
def set_text_with_font(text_frame, text, font_size=16, align=PP_ALIGN.LEFT):
    text_frame.word_wrap = True
    text_frame.clear()
    for line in text.strip().splitlines():
        p = text_frame.add_paragraph()
        p.text = line
        p.font.size = Pt(font_size)
        p.alignment = align

def add_bullets_with_equations(shapes, left, top, width, text, max_image_width=Inches(3.5)):
    """
    Mỗi bullet nằm trong một textbox riêng, cao khoảng 0.8 inch (tương đương ~4 dòng).
    Không có margin trong text_frame. Có hỗ trợ ảnh phương trình (Equation: path).
    """
    eq_pattern = re.compile(r'\(Equation:\s*([^\)]+)\)')
    current_top = top
    bullet_height = Inches(1)
    spacing = Inches(0.2)

    bullets = [line.strip() for line in text.split('\n') if line.strip()]
    for bullet in bullets:
        eq_match = eq_pattern.search(bullet)
        eq_path = None
        if eq_match:
            eq_path = eq_match.group(1).strip()
            bullet = eq_pattern.sub('', bullet).strip()

        # Tạo textbox riêng cho từng bullet
        text_box = shapes.add_textbox(left, current_top, width, bullet_height)
        tf = text_box.text_frame
        tf.word_wrap = True
        tf.clear()

        # Bỏ hết margin nếu muốn
        tf.margin_left = 0
        tf.margin_right = 0
        tf.margin_top = 0
        tf.margin_bottom = 0

        p = tf.add_paragraph()
        p.text = bullet
        p.font.size = Pt(16)

        current_top += bullet_height + spacing

        # Thêm ảnh phương trình nếu có
        if eq_path and os.path.exists(eq_path):
            pic = shapes.add_picture(eq_path, left + (width - max_image_width) / 2, current_top, width=max_image_width)
            pic_height = pic.height.inches
            current_top += Inches(pic_height) + spacing
        elif eq_path:
            print(f"⚠️ Không tìm thấy ảnh phương trình: {eq_path}")

    return current_top


def create_presentation(blocks, pdf_filename, output_file="output.pptx"):
    prs = Presentation()
    blank_slide_layout = prs.slide_layouts[6]  # layout trống

    def add_title_slide(title, subtitle):
        slide = prs.slides.add_slide(prs.slide_layouts[0])
        slide.shapes.title.text = title
        slide.placeholders[1].text = subtitle
        slide.placeholders[1].text_frame.word_wrap = True

    def add_vertical_layout_slide(title, text):
        slide = prs.slides.add_slide(blank_slide_layout)
        shapes = slide.shapes

        title_box = shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(9), Inches(1))
        tf = title_box.text_frame
        tf.clear()
        tf.paragraphs[0].text = title
        tf.paragraphs[0].font.size = Pt(36)
        tf.paragraphs[0].font.bold = True

        left = Inches(0.5)
        top = Inches(1)
        width = Inches(8.5)
        add_bullets_with_equations(shapes, left, top, width, text)

    def add_image_slide(title, text, images):
        slide = prs.slides.add_slide(blank_slide_layout)
        shapes = slide.shapes

        title_box = shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(9), Inches(1))
        tf = title_box.text_frame
        tf.clear()
        tf.paragraphs[0].text = title
        tf.paragraphs[0].font.size = Pt(36)
        tf.paragraphs[0].font.bold = True

        # Text bên trái, có chèn ảnh equation dưới bullet
        left = Inches(0.5)
        top = Inches(1)
        width = Inches(4.3)
        add_bullets_with_equations(shapes, left, top, width, text, max_image_width=Inches(3.5))

        # Ảnh thường (không phải equation) bên phải
        y = 1
        for img in images:
            path = img['path']
            caption = img.get('caption', '')
            if not os.path.exists(path):
                print(f"⚠️ Không tìm thấy ảnh: {path}")
                continue

            pic = slide.shapes.add_picture(path, Inches(5.3), Inches(y), width=Inches(4))
            pic_height = pic.height.inches

            if caption:
                caption_top = y + pic_height + 0.1
                cap_box = slide.shapes.add_textbox(Inches(5.3), Inches(caption_top), Inches(4), Inches(0.6))
                cap_tf = cap_box.text_frame
                cap_tf.clear()
                cap_tf.paragraphs[0].text = caption
                cap_tf.paragraphs[0].font.size = Pt(12)

            y = y + pic_height + 0.5

    # === Duyệt từng block ===
    for i, block in enumerate(blocks):
        title = block.get('heading', '')
        text = block.get('text', '')
        images = []

        # Slide 0 - Title
        if i == 0:
            add_title_slide(title, text)

        # # Slide 1 - Content
        # elif i == 1:
        #     add_vertical_layout_slide(title, text)
        # Slide 1 - Content
        elif i == 1:
            slide = prs.slides.add_slide(blank_slide_layout)
            shapes = slide.shapes
            title_box = shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(9), Inches(1))
            tf = title_box.text_frame
            tf.clear()
            tf.paragraphs[0].text = title
            tf.paragraphs[0].font.size = Pt(36)
            tf.paragraphs[0].font.bold = True

            content_box = shapes.add_textbox(Inches(0.5), Inches(1), Inches(8.5), Inches(5))
            set_text_with_font(content_box.text_frame, text, 24)


        # Các slide còn lại
        else:
            # Lấy ảnh thường (không còn trường equations nữa)
            if block.get('images'):
                for img in block['images']:
                    filename = os.path.basename(img['src'])
                    full_path = os.path.join("/kaggle/working", pdf_filename,"auto/images", filename)
                    caption = img.get('caption', '')
                    images.append({'path': full_path, 'caption': caption})

            if images:
                add_image_slide(title, text, images)
            else:
                add_vertical_layout_slide(title, text)

    prs.save(output_file)
    print(f"✅ File PowerPoint đã được tạo: {output_file}")

def create_slide(pdf_path, markdown_file, pdf_filename, middle_json, content_list_json, image_captions, table_captions, output_file="/kaggle/working/slides/slide_output.pptx"):
    #crop_equations_from_pdf(pdf_path, pdf_filename, middle_json)
    copy_unique_equation_images(pdf_filename, middle_json)
    blocks = parse_markdown(markdown_file, content_list_json, pdf_filename,image_captions, table_captions)
    cleaned_blocks = update_levels_from_heading_text(blocks)
    content_block = create_content_slide(cleaned_blocks)
    cleaned_blocks.insert(1, content_block)

    summarized_blocks = summarize_blocks(cleaned_blocks,2)

    final_blocks  = []
    final_blocks = [cleaned_blocks[0], cleaned_blocks[1]] + summarized_blocks

    for i in range(2, len(final_blocks)):
        block = final_blocks[i]
        if "text" in block:
            lines = block["text"].split('\n')
            updated_lines = []

            for line in lines:
                stripped = line.strip()
                if stripped.startswith("•"):
                    updated_lines.append(stripped)
                elif stripped:
                    updated_lines.append("• " + stripped)
                else:
                    updated_lines.append("")  # dòng trắng giữ nguyên

            block["text"] = "\n".join(updated_lines)

    create_presentation(final_blocks, pdf_filename, output_file)


## Demo Gradio

In [ ]:
%%writefile gradio_app.py
import gradio as gr
import os
import shutil
import pickle
import tempfile
from pdf2image import convert_from_path
import base64
from extract_pdf import extract_pdf
from process_json_data import process_json_data
from create_slide import create_slide
from reranker import get_top_k_docs
from langchain_community.vectorstores import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_ollama import OllamaEmbeddings
import time
from PIL import Image
from langchain_community.embeddings import HuggingFaceEmbeddings
# Gemini API
from google import genai
class GeminiClient:
    def __init__(self, api_key):
        self.client = genai.Client(api_key=api_key)
    def generate_content(self, prompt, model="gemini-2.0-flash"):
        response = self.client.models.generate_content(model=model, contents=prompt)
        return response.text.strip()

# Khởi tạo embedding
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"
gemini_client = GeminiClient(api_key=GEMINI_API_KEY)

# Session state
chat_history = []
global_context = {}

def build_prompt(context, question):
    context_text = "\n".join([doc["document"] for doc in context])
    return f"""
You are given a context that may contain both relevant and irrelevant information. Your task is to answer the question using **only** the most relevant information from the context.

Return your answer in **Markdown format**, suitable for rendering in Gradio using `gr.Markdown(...)`.

If your answer contains mathematical expressions, use **LaTeX syntax** within `$$ ... $$` for block equations, and `$ ... $` for inline math.  
Avoid using any HTML tags like `<sub>`, `<sup>`, or `<b>`.

Use:
- `\\sum`, `\\left(`, `\\right)`, `\\| \\cdot \\|_2^2`, etc., for math formatting.
- Lists or headings for structuring your answer if needed.

Response in **English**
---
**Context**:
{context_text}

**Question**:
{question}
"""


def process_pdf(uploaded_file):
    # Lấy tên gốc
    basename = os.path.basename(uploaded_file.name)

    # Copy giữ tên gốc
    save_dir = "/kaggle/working/uploaded"
    os.makedirs(save_dir, exist_ok=True)
    pdf_path = os.path.join(save_dir, basename)
    shutil.copy(uploaded_file.name, pdf_path)

    # pdf_filename = basename.replace(".pdf", "")

    images = convert_from_path(pdf_path, dpi=300)
    
    html_images = ""

    for idx, img in enumerate(images, start=1):
        # Resize ảnh về 1200px chiều ngang để giữ độ nét, dùng LANCZOS cho chất lượng cao
        target_width = 1200
        img = img.resize((target_width, int(target_width * img.height / img.width)), Image.LANCZOS)
    
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as img_file:
            img.save(img_file.name, "PNG")
            with open(img_file.name, "rb") as f:
                b64 = base64.b64encode(f.read()).decode()
    
                # Thêm số trang trước mỗi ảnh và hiển thị ảnh kích thước nhỏ hơn
                html_images += f'''
                <div style="margin-bottom: 20px;">
                    <div style="font-weight: bold; text-align: center; margin-bottom: 5px;">Page {idx}</div>
                    <img src="data:image/png;base64,{b64}" 
                         style="width:auto; height:auto;" />
                </div>
                '''
    
    html_output = f'''
    <div style="max-height: 800px; overflow: auto; border: 1px solid #ccc; padding: 5px;">
        {html_images}
    </div>
'''

    pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
    #pdf_filename = os.path.basename(saved_path).split(".")[0]
    # jsons_dir = os.path.join("/kaggle/working/output", "jsons")
    # images_dir = os.path.join("/kaggle/working/output", "images")
    # json_folder = os.path.join(jsons_dir, pdf_filename)
    # images_folder = os.path.join(images_dir, pdf_filename)

    root = os.path.join("/kaggle/working", pdf_filename, "auto")
    os.makedirs("/kaggle/working/output/pkl", exist_ok=True)
    pkl_path = os.path.join("/kaggle/working/output/pkl", f"{pdf_filename}.pkl")
    
    markdown_file = os.path.join(root, f"{pdf_filename}.md")
    middle_json = os.path.join(root, f"{pdf_filename}_middle.json")
    content_list_json = os.path.join(root, f"{pdf_filename}_content_list.json")

    images_folder = os.path.join(root, "images")

    if not os.path.exists(root):
        extract_pdf(pdf_path)
        all_docs_summary, image_captions, table_captions = process_json_data(content_list_json, root)
        with open(pkl_path, "wb") as f:
            pickle.dump(all_docs_summary, f)
    else:
        all_docs_summary, image_captions, table_captions = process_json_data(content_list_json, root)
        with open(pkl_path, "wb") as f:
            pickle.dump(all_docs_summary, f)
            
    global_context["pdf_filename"] = pdf_filename
    global_context["pdf_path"] = pdf_path
    global_context["markdown_file"] = markdown_file
    global_context["middle_json"] = middle_json
    global_context["content_list_json"] = content_list_json
    global_context["all_docs_summary"] = all_docs_summary
    global_context["chroma_path"] = f"./chroma_db/{pdf_filename}"
    global_context["image_captions"] = image_captions
    global_context["table_captions"] = table_captions

    return html_output, "PDF processed. You can now chat or create slide."

def create_slide_and_download():
    slide_path = "/kaggle/working/slides/slide_output.pptx"
    os.makedirs("/kaggle/working/slides", exist_ok=True)
    create_slide(global_context["pdf_path"], global_context["markdown_file"],
                 global_context["pdf_filename"],
                 global_context["middle_json"],
                 global_context["content_list_json"],
                 global_context["image_captions"], 
                 global_context["table_captions"])
    
    if os.path.exists(slide_path):
        return slide_path, "✅ Đã tạo xong slide, có thể tải xuống."
    else:
        return None, "❌ Slide chưa được tạo."

def chatbot_response(user_query, top_k):
    all_docs_summary = global_context["all_docs_summary"]
    chroma_path = global_context["chroma_path"]
    #embeddings = OllamaEmbeddings(model="bge-m3", base_url="http://localhost:11434")
    if os.path.exists(chroma_path):
        chroma_db = Chroma(persist_directory=chroma_path, embedding_function=embeddings)
    else:
        chroma_db = Chroma.from_documents(all_docs_summary, embeddings, persist_directory=chroma_path)

    retriever_chroma = chroma_db.as_retriever(search_kwargs={"k": 10})
    bm25_retriever = BM25Retriever.from_documents(all_docs_summary)
    bm25_retriever.k = 10
    ensemble = EnsembleRetriever(retrievers=[bm25_retriever, retriever_chroma], weights=[0.6, 0.4])

    context = get_top_k_docs(user_query, all_docs_summary, k=top_k)
    prompt = build_prompt(context, user_query)
    response = gemini_client.generate_content(prompt)

    chat_history.append((user_query, response))

    # Định dạng Markdown rõ ràng hơn
    formatted_response = f"### Answer:\n{response}"
    context_display = "### Context:\n" + "\n\n".join([f"**Doc {i+1}**:\n{doc['document']}" for i, doc in enumerate(context)])
    
    return formatted_response, chat_history, context_display

# Gradio UI
with gr.Blocks(css="body { background-color: #1e1e1e; color: white; }") as demo:
    gr.Markdown("## 📄 Chat with PDF & Create Slide")

    with gr.Row():
        # Cột trái
        with gr.Column(scale=1):
            pdf_file = gr.File(label="Upload PDF", file_types=[".pdf"])
            pdf_images_html = gr.HTML(label="PDF Pages (Scroll vertical)", elem_id="pdf-images-scroll", 
                                      visible=True)
            status_text = gr.Textbox(label="Status", interactive=False)

            pdf_file.change(fn=process_pdf, inputs=pdf_file, outputs=[pdf_images_html, status_text])

            slide_btn = gr.Button("🛠️ Create Slide")
            slide_download = gr.File(label="📥 Download Slide")
            slide_status = gr.Textbox(label="Slide Status", interactive=False)

            slide_btn.click(fn=create_slide_and_download,
                            outputs=[slide_download, slide_status])
            
            # Chat History chuyển vào cột trái, dưới slide
            chatbox = gr.Chatbot(label="💬 Chat History", height=300)

        # Cột phải
        with gr.Column(scale=1):
            chatbot_in = gr.Textbox(label="Your question")
            top_k_slider = gr.Slider(1, 10, value=3, step=1, label="Top-K Docs")
            chatbot_out = gr.Markdown(label="🧠 AI Response")
            context_out = gr.Markdown(label="📄 Context used")

            chatbot_in.submit(fn=chatbot_response,
                              inputs=[chatbot_in, top_k_slider],
                              outputs=[chatbot_out, chatbox, context_out])

demo.launch(server_name="0.0.0.0", server_port=7860)

In [ ]:
# !ollama pull bge-m3
!python gradio_app.py